Для начала приведем csv файл в корекнтный формат

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


input_file = "/content/drive/MyDrive/final_train.csv"
df = pd.read_csv(input_file)

In [ ]:
df.head()
len(df)

446559

In [ ]:
525806 / 148

3552.7432432432433

In [ ]:
texts = df['text'].values
emojis_data = df['emoji'].factorize()
emojis = emojis_data[0]
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, emojis, test_size=0.1, random_state=42)

In [ ]:
emojis_data[0][:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 0])

In [ ]:
map = {}
for i in range(len(emojis_data[0])):
    em = emojis_data[0][i]
    if em not in map:
        map[em] = 1
    else:
        map[em] += 1

print(len(map))

62


In [ ]:
def emoji_to_num(emoji):
  for index, elem in enumerate(emojis_data[1]):
    if elem == emoji:
      return index

def num_to_emoji(num):
  return emojis_data[1][num]

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch


tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
tokenizer.truncation_side = 'left'
bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [ ]:
def tokenize_data(texts, tokenizer, max_len=24):
    return tokenizer(
        texts.tolist(),
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )

In [ ]:
train_encodings = tokenize_data(train_texts, tokenizer)
val_encodings = tokenize_data(val_texts, tokenizer)

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)


Создаем пайторч датасет

In [ ]:
import torch

class EmojiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx].clone().detach()
        return item

train_dataset = EmojiDataset(train_encodings, train_labels)
val_dataset = EmojiDataset(val_encodings, val_labels)


In [ ]:
def tokenize_data2(texts, tokenizer, max_len=24):
    return tokenizer(
        texts,
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
test = ["здарова sdgd sdgdfg",
        "привет"]
emb_test = tokenize_data2(test, tokenizer)
emb_test2 = bert(**emb_test)
emb_test2.last_hidden_state.shape

torch.Size([2, 13, 768])

Создаем модель

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, inputs, targets):
        probs = F.softmax(inputs, dim=-1)
        # Достаем нужные вероятности для классов, указанных в targets
        targets_one_hot = F.one_hot(targets, num_classes=inputs.shape[-1])
        probs = torch.sum(probs * targets_one_hot, dim=-1)

        # Вычисляем фокусирующий параметр
        focal_weight = (1 - probs) ** self.gamma

        # Стандартная кросс-энтропийная ошибка
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        loss = focal_weight * ce_loss

        if self.alpha is not None:
            alpha_weight = self.alpha.gather(0, targets)
            loss = alpha_weight * loss

        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss


find alphas

In [ ]:
# num_classes = 148
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# alpha = torch.zeros(num_classes).to(device)
# for i in range(len(alpha)):
#   alpha[i] = 2.5

# frequent = ['😍', '😊', '❤', '😂', '😁', '😅', '👍', '🤣', '😭']
# common = ['😢', '😀', '🤷', '🤗', '😌', '🥺', '🤩', '🥲', '💋', '😎', '🏼', '🏻', '😉', '🤦', '😏', '🥹', '😋', '🤤', '💚', '🤔', '✨', '😃', '🥰', '😔', '🙈', '😄', '😳', '👌', '🙏', '🔥', '😆', '☺', '😱', '🙂', '🫶', '🏆', '🥳', '👀']

# for el in frequent:
#   num = emoji_to_num(el)
#   alpha[num] = 0.1

# for el in common:
#   num = emoji_to_num(el)
#   alpha[num] = 0.7

# for i in range(148):
#   print(num_to_emoji(i), alpha[i])


num_classes = 62
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

alpha = torch.zeros(num_classes).to(device)
for i in range(len(alpha)):
  alpha[i] = 3500 / map[i]

# for i in range(148):
#   print(num_to_emoji(i), alpha[i])


alpha2 = torch.ones(num_classes).to(device)

alpha3 = torch.sqrt(alpha)



In [ ]:
!python -m spacy download ru_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 78.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import torch
import torch.nn as nn
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

def tokenize_data(texts):
    return [word_tokenize(text.lower()) for text in texts]

train_tokenized = tokenize_data(train_texts)
val_tokenized = tokenize_data(val_texts)

# Обучаем модель Word2Vec на наших данных
model = Word2Vec(sentences=train_tokenized, vector_size=100, window=5, min_count=1, workers=4)

def texts_to_vectors(texts, model):
    vectors = []
    for text in texts:
        vector = sum([model.wv[word] for word in text if word in model.wv])
        vectors.append(vector / len(text) if len(text) > 0 else vector)
    return torch.tensor(vectors)

train_vectors = texts_to_vectors(train_tokenized, model)
val_vectors = texts_to_vectors(val_tokenized, model)

train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

class EmojiDataset(torch.utils.data.Dataset):
    def __init__(self, vectors, labels):
        self.vectors = vectors
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'vector': self.vectors[idx], 'labels': self.labels[idx]}

train_dataset = EmojiDataset(train_vectors, train_labels)
val_dataset = EmojiDataset(val_vectors, val_labels)

class EmojiClassifier(nn.Module):
    def __init__(self, input_size, num_labels, alpha):
        super(EmojiClassifier, self).__init__()
        self.alpha = alpha
        self.classifier = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_labels)
        )

    def forward(self, vector, labels=None):
        logits = self.classifier(vector)

        if labels is not None:
            loss_fct = FocalLoss(gamma=1.0, alpha=self.alpha)
            loss = loss_fct(logits, labels)
            return loss, logits
        return logits

In [ ]:
from tqdm import tqdm
import torch
from tqdm import tqdm
import numpy as np

from torch.utils.tensorboard import SummaryWriter

def train_model(model, train_loader, val_loader, epochs=10):

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        loop = tqdm(train_loader, leave=True)

        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs[0]
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            loop.set_description(f'Epoch [{epoch+1}/{epochs}]')
            loop.set_postfix(loss=loss.item())

        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch + 1}, Loss: {avg_loss}')

        model.eval()
        correct = 0
        total = 0
        correct_top10 = 0
        class_total = np.zeros(num_classes)
        class_correct = np.zeros(num_classes)
        class_predicted = np.zeros(num_classes) # num of elements that were predicted to this class


        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating"):
                batch = {k: v.to(device) for k, v in batch.items()}
                logits = model(**batch)[1]
                predictions = torch.argmax(logits, dim=-1)
                correct += (predictions == batch['labels']).sum().item()
                total += len(batch['labels'])

                top10_preds = torch.topk(logits, k=10, dim=-1).indices
                correct_top10 += sum([batch['labels'][i] in top10_preds[i] for i in range(len(batch['labels']))])

                for i in range(len(batch['labels'])):
                    label = batch['labels'][i].item()
                    pred = predictions[i].item()
                    class_predicted[pred] += 1
                    class_total[label] += 1
                    if label == pred:
                        class_correct[label] += 1


        top1_acc = correct / total * 100
        top10_acc = correct_top10 / total * 100


        print(f'Validation Accuracy (Top-1): {top1_acc:.2f}%')
        print(f'Validation Accuracy (Top-10): {top10_acc:.2f}%')

        for i in range(num_classes):
            if class_total[i] > 0 and class_predicted[i] > 0:
                recall = 100 * class_correct[i] / class_total[i]
                prec = 100 * class_correct[i] / class_predicted[i]
                print(f'Class {i} Recall: {recall:.2f}% ({int(class_correct[i])}/{int(class_total[i])})      Precision: {prec:.2f}% ({int(class_correct[i])}/{int(class_predicted[i])})') # num of elements from this class that were predicted correctly
            else:
                print(f'Class {i} Accuracy: N/A (нет примеров для этого класса) class_predicted {class_predicted[i]}, class_total {class_total[i]}, class_correct {class_correct[i]}')


train_model(model, train_loader, val_loader)

In [ ]:
import torch

def tokenize_data(texts, max_len=24):


train_encodings = tokenize_data(train_texts, tokenizer)
val_encodings = tokenize_data(val_texts, tokenizer)


train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)


class EmojiDataset(torch.utils.data.Dataset):
    def __init__(self, text, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx].clone().detach()
        return item

train_dataset = EmojiDataset(train_encodings, train_labels)
val_dataset = EmojiDataset(val_encodings, val_labels)


In [ ]:
class USEClassifier(nn.Module):
    def __init__(self, num_labels, alpha=None):
        super(USEClassifier, self).__init__()
        self.num_labels = num_labels
        self.alpha = alpha
        self.classifier = nn.Linear(512, num_labels)  # 512 - размер эмбеддинга от USE

    def get_use_embeddings(self, texts):
        # Преобразуем текст в эмбеддинги с помощью USE
        embeddings = use_embedder(texts)  # Получаем эмбеддинги через USE
        return torch.tensor(embeddings.numpy())  # Преобразуем их в тензор PyTorch

    def forward(self, texts, labels=None):
        # Получаем эмбеддинги для текстов
        use_embeddings = self.get_use_embeddings(texts)

        # Прямой проход через классификатор
        logits = self.classifier(use_embeddings)

        if labels is not None:
            # Использование FocalLoss для дисбаланса классов
            loss_fct = FocalLoss(gamma=1.0, alpha=self.alpha)
            loss = loss_fct(logits, labels)
            return loss, logits

        return logits



In [ ]:
import torch.nn as nn
from transformers import BertModel
from transformers import AutoModel
from peft import LoraConfig, get_peft_model
import torch.nn.functional as F

class EmojiClassifier3(nn.Module):
    def __init__(self, num_labels, alpha):
        super(EmojiClassifier3, self).__init__()
        self.alpha = alpha
        self.bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
        for param in self.bert.parameters():
            param.requires_grad = False
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_labels)
        )

    def forward(self, input_ids, attention_mask, token_type_ids, labels=None): # тут посмотреть
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.pooler_output
        logits = self.classifier(cls_output)

        if labels is not None:
            loss_fct = FocalLoss(gamma=1.0, alpha=self.alpha)
            loss = loss_fct(logits, labels)
            #print(loss)
            return loss, logits
        return logits

In [ ]:
import torch.nn as nn
from transformers import BertModel

class EmojiClassifier(nn.Module):
    def __init__(self, num_labels):
        super(EmojiClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
        for param in self.bert.parameters():
            param.requires_grad = False
        self.classifier = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_labels)
        )

    def forward(self, input_ids, attention_mask, token_type_ids, labels=None): # тут посмотреть
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.pooler_output
        logits = self.classifier(cls_output)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return loss, logits
        return logits

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import LambdaLR


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = EmojiClassifier2(num_labels=len(df['emoji'].unique()), alpha=alpha3).to(device)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)

optimizer = AdamW(model.parameters(), lr=2e-4)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def lr_lambda(current_step: int):
    warmup_steps = 200  # Количество шагов для warmup
    if current_step < warmup_steps:
        return float(current_step) / float(max(1, warmup_steps))
    return 1.0

scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)


In [ ]:
from tqdm import tqdm
import torch
from tqdm import tqdm
import numpy as np

from torch.utils.tensorboard import SummaryWriter

def train_model(model, train_loader, val_loader, epochs=10):
    # Инициализация TensorBoard
    writer = SummaryWriter()

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        loop = tqdm(train_loader, leave=True)

        for batch in loop:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs[0]
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

            loop.set_description(f'Epoch [{epoch+1}/{epochs}]')
            loop.set_postfix(loss=loss.item())

        # Логируем тренировочную потерю в TensorBoard
        avg_loss = total_loss / len(train_loader)
        writer.add_scalar('Loss/train', avg_loss, epoch)
        print(f'Epoch {epoch + 1}, Loss: {avg_loss}')

        model.eval()
        correct = 0
        total = 0
        correct_top10 = 0
        class_total = np.zeros(num_classes)
        class_correct = np.zeros(num_classes)
        class_predicted = np.zeros(num_classes) # num of elements that were predicted to this class


        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating"):
                batch = {k: v.to(device) for k, v in batch.items()}
                logits = model(**batch)[1]
                predictions = torch.argmax(logits, dim=-1)
                correct += (predictions == batch['labels']).sum().item()
                total += len(batch['labels'])

                top10_preds = torch.topk(logits, k=10, dim=-1).indices
                correct_top10 += sum([batch['labels'][i] in top10_preds[i] for i in range(len(batch['labels']))])

                for i in range(len(batch['labels'])):
                    label = batch['labels'][i].item()
                    pred = predictions[i].item()
                    class_predicted[pred] += 1
                    class_total[label] += 1
                    if label == pred:
                        class_correct[label] += 1


        top1_acc = correct / total * 100
        top10_acc = correct_top10 / total * 100

        # Логируем в TensorBoard метрики валидации
        writer.add_scalar('Accuracy/Top-1', top1_acc, epoch)
        writer.add_scalar('Accuracy/Top-10', top10_acc, epoch)

        print(f'Validation Accuracy (Top-1): {top1_acc:.2f}%')
        print(f'Validation Accuracy (Top-10): {top10_acc:.2f}%')

        for i in range(num_classes):
            if class_total[i] > 0 and class_predicted[i] > 0:
                recall = 100 * class_correct[i] / class_total[i]
                prec = 100 * class_correct[i] / class_predicted[i]
                print(f'Class {i} Recall: {recall:.2f}% ({int(class_correct[i])}/{int(class_total[i])})      Precision: {prec:.2f}% ({int(class_correct[i])}/{int(class_predicted[i])})') # num of elements from this class that were predicted correctly
            else:
                print(f'Class {i} Accuracy: N/A (нет примеров для этого класса) class_predicted {class_predicted[i]}, class_total {class_total[i]}, class_correct {class_correct[i]}')

    # Закрываем TensorBoard
    writer.close()



train_model(model, train_loader, val_loader)

Epoch [1/10]: 100%|██████████| 785/785 [17:22<00:00,  1.33s/it, loss=2.43]


Epoch 1, Loss: 2.354225932868423


Validating: 100%|██████████| 698/698 [00:59<00:00, 11.68it/s]


Validation Accuracy (Top-1): 11.43%
Validation Accuracy (Top-10): 48.30%
Class 0 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 2011.0, class_correct 0.0
Class 1 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 389.0, class_correct 0.0
Class 2 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 1118.0, class_correct 0.0
Class 3 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 351.0, class_correct 0.0
Class 4 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 1531.0, class_correct 0.0
Class 5 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 338.0, class_correct 0.0
Class 6 Recall: 100.00% (5104/5104)      Precision: 11.43% (5104/44656)
Class 7 Accuracy: N/A (нет примеров для этого класса) class_predicted 0.0, class_total 3699.0, class_correct 0.0
Class 8 Accuracy: N/A (нет примеров для этого класса) class_predict

Epoch [2/10]:   8%|▊         | 60/785 [01:19<16:04,  1.33s/it, loss=2.43]


KeyboardInterrupt: 

In [ ]:
import torch
from tqdm import tqdm
import numpy as np

def val_model(model, train_loader, val_loader, epochs=10):

        # Валидация
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        correct_top10 = 0

        # Количество классов (например, 150 классов)
        num_classes = 148
        class_correct = np.zeros(num_classes)  # Счётчики правильных предсказаний для каждого класса
        class_total = np.zeros(num_classes)    # Общее количество примеров для каждого класса
        class_predicted = np.zeros(num_classes) # num of elements that were predicted to this class

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validating"):
                batch = {k: v.to(device) for k, v in batch.items()}
                logits = model(**batch)[1]
                predictions = torch.argmax(logits, dim=-1)

                # Обновление счётчиков по классам
                for i in range(len(batch['labels'])):
                    label = batch['labels'][i].item()
                    pred = predictions[i].item()
                    class_predicted[pred] += 1
                    class_total[label] += 1
                    if label == pred:
                        class_correct[label] += 1

        print(f'Validation Accuracy (Top-10): {top10_acc:.2f}%')

        # Вычисление accuracy по каждому классу
        for i in range(num_classes):
            if class_total[i] > 0 and class_predicted[i] > 0:
                recall = 100 * class_correct[i] / class_total[i]
                prec = 100 * class_correct[i] / class_predicted[i]
                print(f'Class {i} Recall: {recall:.2f}% ({int(class_correct[i])}/{int(class_total[i])})      Precision: {prec:.2f}% ({int(class_correct[i])}/{int(class_predicted[i])})') # num of elements from this class that were predicted correctly
            else:
                print(f'Class {i} Accuracy: N/A (нет примеров для этого класса) class_predicted {class_predicted[i]}, class_total {class_total[i]}, class_correct {class_correct[i]}')

val_model(model, train_loader, val_loader)

In [ ]:
bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")





In [ ]:

for param in bert.parameters():
    param.requires_grad = False

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)


print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

In [ ]:
bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],
    bias="none"
)

bert = get_peft_model(bert, lora_config)

for param in bert.parameters():
    param.requires_grad = False


for name, param in bert.named_parameters():
            if 'lora' in name:
                param.requires_grad = True

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)


print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 178148352
Trainable parameters: 294912


In [ ]:
bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value", "key"],
    bias="none"
)

bert = get_peft_model(bert, lora_config)

for param in bert.parameters():
    param.requires_grad = False


for name, param in bert.named_parameters():
            if 'lora' in name:
                param.requires_grad = True

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)


print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 178295808
Trainable parameters: 442368


In [ ]:
bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["query", "value", "key"],
    bias="none"
)

bert = get_peft_model(bert, lora_config)

for param in bert.parameters():
    param.requires_grad = False


for name, param in bert.named_parameters():
            if 'lora' in name:
                param.requires_grad = True

total_params = sum(p.numel() for p in bert.parameters())
trainable_params = sum(p.numel() for p in bert.parameters() if p.requires_grad)


print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 179622912
Trainable parameters: 1769472
